In [2]:
import pandas as pd
import json
from sqlalchemy import create_engine

# Caricamento delle  le variabili d'ambiente dal file config.json
config_path = 'D:/Reepo_Git/Google_play_store/'
with open(config_path + 'config.json', 'r') as file:
    config = json.load(file)

clean_data_path = config['clean_data_path']
db_connection_string = config['db_connection_string']
schema = config['schema']

google_data_name = clean_data_path + "clean_googleplaystore.csv"
google_data_review_name = clean_data_path + "clean_googleplaystore_user_reviews"



class SQLHandler:
    """Classe per gestire il caricamento dei dati in un database SQL."""

    def __init__(self, original_data, conn_string, schema):
        self.original_data = original_data
        self.conn_string = conn_string
        self.schema = schema
        # creazione dell' istanza 'engine' per il database
        self.engine = create_engine(self.conn_string)

    def load_original_data(self, table_name):
        try:
            self.original_data.to_sql(table_name, 
                                        self.engine, 
                                        schema=self.schema, 
                                        if_exists='replace', 
                                        index=False)
        except Exception as error:
            print(f"Error loading original data: {error}")


    def close_connection(self):
        self.engine.dispose()

# Crea la connessione e popola il database con i dati 
def connection(conn_string, clean_data_path, schema, table_name):
    try:
        original_data = pd.read_csv(clean_data_path)
        conn_handler = SQLHandler(original_data, conn_string, schema)
        conn_handler.load_original_data(table_name)
        conn_handler.close_connection()
    except Exception as error:
        print(f"Error in connection function: {error}")

# Carica i dati del file clean_googleplaystore     
connection(conn_string= db_connection_string, 
            clean_data_path= google_data_name,  
            schema= schema,
            table_name= 'googleplaystore')

# Carica i dati del file clean_googleplaystore_user_reviews
connection(conn_string= db_connection_string, 
            clean_data_path= google_data_name,  
            schema= schema,
            table_name= 'googleplaystore_user_reviews')